In [23]:
%load_ext autoreload
%autoreload 2

from __future__ import division

import os
import os.path
import math
import pickle
import multiprocessing
import collections
import re
import csv

import numpy as np
import Bio.ExPASy.Prosite as BEP
import praline
import praline.container
import mapraline.component
import mapraline.prosite
from praline.core import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
INPUT_DIR = 'RV100/'
TRACK_ID_BASE = "mapraline.track.PrositePatternAnnotation"
FMT_TRACK_ID = "{0}_{1}"
ALPHABET = mapraline.ALPHABET_PROSITE
PROSITE_PATH = '/Users/maurits/Downloads/prosite.dat'

In [3]:
def read_patterns(path):
    patterns = []

    with file(path, 'r') as fi:
        for record in BEP.parse(fi):
            pattern = record.pattern.rstrip('.')
            if len(pattern):
                patterns.append(pattern)
    
    return patterns

In [7]:
# Read all patterns from our local prosite db.
patterns = read_patterns(PROSITE_PATH)

seq_sets = []
for path, dirnames, filenames in os.walk(INPUT_DIR):
    for filename in filenames:
        if not filename.endswith('.tfa'):
            continue
            
        path = os.path.join(INPUT_DIR, filename)
        try:
            seq_set = praline.load_sequence_fasta(path, praline.container.ALPHABET_AA)
        except praline.AlphabetError:
            continue
        
        seq_sets.append((filename, seq_set))

218


In [8]:
seq_sets = [(f, s) for f, s in seq_sets if len(s) < 150]

196


In [10]:
family_names = [os.path.splitext(f)[0] for f, s in seq_sets]

In [11]:
regexes = []
for pattern in patterns:
    re = mapraline.prosite.pattern_to_re(pattern)
    regexes.append((pattern, re))

In [12]:
def get_raw_sequences(seqs):
    raw_seqs = []
    for seq in seqs:
        track = seq.get_track(praline.container.TRACK_ID_INPUT)
        
        indices = track.values
        sym_list = []
        for n in xrange(indices.shape[0]):
            sym_list.append(track.alphabet.index_to_symbol(indices[n]))
        raw_seqs.append("".join(sym_list))
    
    return raw_seqs


In [13]:
raw_seq_sets = [get_raw_sequences(seq_set) for filename, seq_set in seq_sets]

In [14]:
pattern_to_match_sets = {}
for pattern, regex in regexes:
    match_sets = []
    for seq_set in raw_seq_sets:
        match_set = []
        for seq in seq_set:
            matches = list(regex.finditer(seq, overlapped=True))
            match_set.append(matches)
        match_sets.append(match_set)
    pattern_to_match_sets[pattern] = match_sets

In [15]:
count_per_set = collections.defaultdict(lambda: 0)
count_per_seq = collections.defaultdict(lambda: 0)
for pattern, match_sets in pattern_to_match_sets.iteritems():
    for i, match_set in enumerate(match_sets):
        for j, matches in enumerate(match_set):
            count_per_set[i] += len(matches)
            count_per_seq[i, j] += len(matches)

In [16]:
counts = np.array(count_per_set.values())
lengths = np.array([sum(len(seq) for seq in set_) for set_ in raw_seq_sets])

In [17]:
normalized_counts = counts / lengths

In [18]:
normalized_counts.min(), normalized_counts.max(), normalized_counts.mean()

(0.030540179423554113, 0.0942772080714522, 0.052956166529453924)

In [19]:
normalized_counts.argmin(), normalized_counts.argmax(), normalized_counts.mean()

(73, 105, 0.052956166529453924)

In [20]:
pattern_to_covered = {}
for pattern, match_sets in pattern_to_match_sets.iteritems():
    covered_sets = []
    for i, match_set in enumerate(match_sets):
        num_covered = 0
        for matches in match_set:
            if len(matches) > 0:
                num_covered += 1
        if (num_covered / len(match_set)) > 0.5:
            covered_sets.append(i)
    
    if len(covered_sets) > 0:
        pattern_to_covered[pattern] = covered_sets
            
            

In [21]:
covered_by = collections.defaultdict(set)
for pattern, families in pattern_to_covered.iteritems():
    for family in families:
        covered_by[family_names[family]].add(pattern)

In [22]:
with open('./commands.txt', 'w') as fo:
    for alpha in [0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 100]:
        for family, patterns in covered_by.iteritems():
            filename = "{}.tfa".format(family)
            fmt = "mapraline input/{0} output/{1} --motif-match-score {2} {3}"
            pat_str = " ".join("-p '{0}'".format(x) for x in patterns)

            base_fname, ext = os.path.splitext(filename)
            out_filename = '{0}_alpha_{1}.aln'.format(base_fname, alpha)
            print >>fo, fmt.format(filename, out_filename, alpha, pat_str)
    

In [27]:
with open('./s3_table.csv', 'wb') as fo:
    writer = csv.DictWriter(fo, dialect='excel', fieldnames=['BB4_family', 'pattern'])
    writer.writeheader()
    for family, patterns in sorted(covered_by.items(), key=lambda x: int(x[0][4:])):
        pat_list = list(patterns)
        writer.writerow({'BB4_family': family, 'pattern': pat_list[0]})
        for pattern in pat_list[1:]:
            writer.writerow({'BB4_family': '', 'pattern': pattern})
        

In [29]:
with open('./family_patterns.pickle', 'wb') as fo:
    pickle.dump(covered_by, fo)